<html>
<p style="font-size: 300%; font-weight: bold; color: red; text-align: center; line-height: 120%">
Please "Close and Halt" this notebook after use!
</p>
</html>

# Introduction

**JavaScript Object Notation** or **JSON** is an open-standard file format that uses human-readable text to transmit data objects consisting of attribute–value pairs and array data types (or any other serializable value). It is a very common data format used for asynchronous browser–server communication, including as a replacement for XML in some AJAX-style systems.

JSON is a language-independent data format. It was derived from JavaScript, but as of 2017 many programming languages include code to generate and parse JSON-format data. The official Internet media type for JSON is `application/json`. JSON filenames use the extension `.json`.

## Tools
We focus on three tools:
1. Python package `json` https://docs.python.org/2/library/json.html
2. UNIX command line tool `jq` https://stedolan.github.io/jq/
3. PySpark https://spark.apache.org

## References
- Tweet data dictionaries https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object
- https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/intro-to-tweet-json.html
- Parsing JSON with jq http://www.compciv.org/recipes/cli/jq-for-parsing-json/

# Data Set
We use an small subset of a Twitter data-set, which contains over 170,000,000 tweets collected during 3 months leading up to the 2012 presidential elections. The original data files are posted at https://old.datahub.io/dataset/twitter-2012-presidential-election



In [47]:
%%sh
head -1 twitter200.json | jq '.'

{
  "in_reply_to_user_id_str": null,
  "id_str": "246058156919238656",
  "text": "Anderson Silva vs Stephen bonner is like sending a man to murders row after he saved the president 5 years earlier",
  "geo": null,
  "retweeted": false,
  "in_reply_to_status_id": null,
  "created_at": "Thu Sep 13 01:30:10 +0000 2012",
  "source": "web",
  "entities": {
    "urls": [],
    "user_mentions": [],
    "hashtags": []
  },
  "contributors": null,
  "place": null,
  "favorited": false,
  "coordinates": null,
  "retweet_count": 0,
  "truncated": false,
  "in_reply_to_status_id_str": null,
  "user": {
    "id_str": "29324689",
    "is_translator": false,
    "verified": false,
    "favourites_count": 9,
    "geo_enabled": false,
    "profile_use_background_image": false,
    "profile_image_url": "http://a0.twimg.com/profile_images/2581517833/1y7d6c4gxsublrfhmium_normal.jpeg",
    "profile_text_color": "666666",
    "profile_background_image_url": "http://a0.twimg.com/profile_background_images/595

# Python


In [49]:
import json
import os, sys

In [51]:
MAX_LINES = 10
with open('twitter200.json') as io:
    line = io.readline()
    while line and MAX_LINES>0:
        row = json.loads(line.strip())
        print row.keys()
        
        # end of while-body
        MAX_LINES -= 1
        line = io.readline()
        

[u'favorited', u'in_reply_to_user_id', u'contributors', u'entities', u'text', u'created_at', u'truncated', u'retweeted', u'in_reply_to_status_id_str', u'coordinates', u'id', u'source', u'in_reply_to_status_id', u'place', u'id_str', u'in_reply_to_screen_name', u'retweet_count', u'geo', u'in_reply_to_user_id_str', u'user']
[u'contributors', u'truncated', u'text', u'in_reply_to_status_id', u'id', u'source', u'retweeted', u'coordinates', u'entities', u'in_reply_to_screen_name', u'in_reply_to_user_id', u'retweet_count', u'id_str', u'favorited', u'user', u'geo', u'in_reply_to_user_id_str', u'possibly_sensitive', u'created_at', u'possibly_sensitive_editable', u'in_reply_to_status_id_str', u'place']
[u'contributors', u'truncated', u'text', u'in_reply_to_status_id', u'id', u'source', u'retweeted', u'coordinates', u'entities', u'in_reply_to_screen_name', u'in_reply_to_user_id', u'retweet_count', u'id_str', u'favorited', u'user', u'geo', u'in_reply_to_user_id_str', u'possibly_sensitive', u'create

In [52]:
row

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Thu Sep 13 01:30:11 +0000 2012',
 u'entities': {u'hashtags': [],
  u'urls': [],
  u'user_mentions': [{u'id': 34993057,
    u'id_str': u'34993057',
    u'indices': [3, 15],
    u'name': u'JustMePammy',
    u'screen_name': u'JustMePammy'},
   {u'id': 376514554,
    u'id_str': u'376514554',
    u'indices': [24, 36],
    u'name': u'@bornAmerica',
    u'screen_name': u'BornAmerica'}]},
 u'favorited': False,
 u'geo': None,
 u'id': 246058158190129153,
 u'id_str': u'246058158190129153',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'place': None,
 u'retweet_count': 2,
 u'retweeted': False,
 u'retweeted_status': {u'contributors': None,
  u'coordinates': None,
  u'created_at': u'Thu Sep 13 01:25:48 +0000 2012',
  u'entities': {u'hashtags': [],
   u'urls': [],
   u'user_mentions': [{u'id': 376514554,
     u'

In [53]:
row['text']

u'RT @JustMePammy: &gt;RT @BornAmerica: The USA is a Republic.'

In [54]:
print row['text'], row['user']['id'], row['user']['name']

RT @JustMePammy: &gt;RT @BornAmerica: The USA is a Republic. 376514554 @bornAmerica


## Table of users and web-sites

Let's create a table of users and web-sites they are tweeted about. This may be useful to create a graph between web-sites determined by the number of shared uses. Eventually, we could also include a sentiment analysis of the tweet.

In [55]:
### Let's start with the core loop
MAX_LINES = 7
with open('twitter200.json') as io:
    line = io.readline()
    while line and MAX_LINES>0:
        row = json.loads(line.strip())
        for u in row['entities']['urls']:
            print row['user']['id'], u['expanded_url'].split('/', 3)[2]
        
        # end of while-body
        MAX_LINES -= 1
        line = io.readline()

61914925 yfrog.com
340243491 www.egyptbulletin.com
606657776 QualityPetWorld.com
66700814 on.cnn.com


In [56]:
row

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Thu Sep 13 01:30:11 +0000 2012',
 u'entities': {u'hashtags': [],
  u'urls': [],
  u'user_mentions': [{u'id': 133110529,
    u'id_str': u'133110529',
    u'indices': [3, 12],
    u'name': u'Karl Welzein',
    u'screen_name': u'DadBoner'}]},
 u'favorited': False,
 u'geo': None,
 u'id': 246058158043308033,
 u'id_str': u'246058158043308033',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'place': None,
 u'retweet_count': 275,
 u'retweeted': False,
 u'retweeted_status': {u'contributors': None,
  u'coordinates': None,
  u'created_at': u'Wed Sep 12 19:18:42 +0000 2012',
  u'entities': {u'hashtags': [], u'urls': [], u'user_mentions': []},
  u'favorited': False,
  u'geo': None,
  u'id': 245964674347196419,
  u'id_str': u'245964674347196419',
  u'in_reply_to_screen_name': None,
  u'in_reply_to_status_id': No

# JQ

- https://stedolan.github.io/jq/
- Parsing JSON with jq http://www.compciv.org/recipes/cli/jq-for-parsing-json/

jq is like sed for JSON data - you can use it to slice and filter and map and transform structured data with the same ease that sed, awk, grep and friends let you play with text.

jq is written in portable C, and it has zero runtime dependencies. You can download a single binary, scp it to a far away machine of the same type, and expect it to work.

jq can mangle the data format that you have into the one that you want with very little effort, and the program to do so is often shorter and simpler than you’d expect.

In [57]:
%%sh
head -20 < twitter200.json | jq -c '{user_name: .user.name, user_id: .user.id, links: [.entities.urls[].expanded_url]}' | head -20


{"user_name":"Greg S.","user_id":29324689,"links":[]}
{"user_name":"Kevon O ","user_id":61914925,"links":["http://yfrog.com/hswxhzmj"]}
{"user_name":"Egypt Bulletin","user_id":340243491,"links":["http://www.egyptbulletin.com/80466_en"]}
{"user_name":"Erin","user_id":129869846,"links":[]}
{"user_name":"Quality Pet World","user_id":606657776,"links":["http://QualityPetWorld.com/news/detail.php?fid=35"]}
{"user_name":"Claire Barrett","user_id":66700814,"links":["http://on.cnn.com/cnndcl2"]}
{"user_name":"Bran","user_id":34483672,"links":[]}
{"user_name":"shipster ","user_id":173871849,"links":[]}
{"user_name":"Blackk Dynamite®","user_id":614012051,"links":[]}
{"user_name":"@bornAmerica","user_id":376514554,"links":[]}
{"user_name":"Paul Matreselva ","user_id":501350941,"links":["http://mmajunkie.com/news/30610/usa-today-anderson-silva-vs-stephan-bonnar-headlines-ufc-153.mma"]}
{"user_name":"careerscallcenter","user_id":562488577,"links":["http://www.careerscallcenter.com/usa/illinois/maco

In [58]:
%%sh
cat twitter200.json | jq -c '.geo' | sort | uniq -c | sort -rn 

    198 null
      1 {"type":"Point","coordinates":[50.0866999,14.4188868]}
      1 {"type":"Point","coordinates":[-25.4783811,-49.2938965]}


In [59]:
%%sh
cat twitter1k.json | grep -v '"geo":null' | wc -l 

9


# Spark


In [1]:
import sys, os
sys.path.insert(0, '/usr/hdp/current/spark2-client/python')

In [2]:
from pyspark import SparkConf, SparkContext

In [ ]:
sc.stop()

In [5]:
conf = SparkConf().setMaster('yarn').setAppName("TwitterData")
conf.set("spark.shuffle.service.enabled", "true") \
    .set("spark.dynamicAllocation.enabled", "true") \
    .set("spark.dynamicAllocation.executorIdleTimeout", "120s") \
    .set("spark.dynamicAllocation.initialExecutors", 4)
if not 'sc' in globals():
    sc = SparkContext(conf=conf)
print sc.version
print "Spark UI at http://arc.insight.gsu.edu:8088/proxy/"+sc.applicationId

2.2.0.2.6.3.0-235
Spark UI at http://arc.insight.gsu.edu:8088/proxy/application_1514672663667_0465


## Read Twitter Data

The election data are on the ARC cluster's HDFS in `/data/election2012`

In [6]:
%%sh
hdfs dfs -ls /data/election2012 | head -4

Found 171 items
-rw-r--r--   3 molnar hdfs  476352240 2018-02-10 10:25 /data/election2012/cache-0.json.gz
-rw-r--r--   3 molnar hdfs  495723894 2018-02-10 10:25 /data/election2012/cache-1000000.json.gz
-rw-r--r--   3 molnar hdfs  474653135 2018-02-10 10:25 /data/election2012/cache-10000000.json.gz


In [12]:
# All records
## rdd_all = sc.textFile('/data/election2012')

# only a batch of 1,000,000
rdd = sc.textFile('/data/election2012/cache-10000000.json.gz')
rdd.take(1)

[u'{"in_reply_to_status_id_str":null,"text":"Put the *media* into #mixedmedia projects, from copper fabric to luminescent tape: http:\\/\\/t.co\\/BRm1dMgH","created_at":"Sun Sep 23 13:51:57 +0000 2012","truncated":false,"possibly_sensitive":false,"in_reply_to_user_id_str":null,"contributors":null,"retweeted":false,"possibly_sensitive_editable":true,"coordinates":null,"in_reply_to_status_id":null,"source":"web","entities":{"urls":[{"display_url":"wp.me\\/p2H1i-1uD","indices":[83,103],"url":"http:\\/\\/t.co\\/BRm1dMgH","expanded_url":"http:\\/\\/wp.me\\/p2H1i-1uD"}],"user_mentions":[],"hashtags":[{"text":"mixedmedia","indices":[21,32]}]},"place":null,"geo":null,"in_reply_to_user_id":null,"favorited":false,"retweet_count":0,"user":{"created_at":"Wed Mar 11 18:07:07 +0000 2009","profile_sidebar_border_color":"E64615","profile_image_url_https":"https:\\/\\/si0.twimg.com\\/profile_images\\/289384511\\/pencil_tip_normal.jpg","default_profile_image":false,"friends_count":1693,"url":"http:\\/\\

Rather than decoding each line from JSON to Python Dict we may just look for substrings. In this case we can filter out all tweets without a URLs.

In [13]:
rdd_with_urls = rdd.filter(lambda s: not ',"urls":[]' in s)

In [14]:
rdd_with_urls.count()

986575

Counting the entire data set takes a bit over 3 minutes on the cluster (Don't try this at home!)

In [15]:
# rdd_all = sc.textFile('/data/election2012')
# rdd_all.count()

170515361

## Spark SQL
We can also read the data set into a DataFrame. However, that's not necessarily the best thing to do. When reading JSON files, Spark needs to load the entire dataset to infer a schema. This can take a while (on the cluster about 2 to 3 minutes).  


In [16]:
from pyspark.sql import SQLContext

sqlCtx = SQLContext(sc)

In [18]:
#df = sqlCtx.read.json('/data/election2012')
#df.printSchema()

root
 |-- contributors: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |  

Because some of the columns in the DataFrame are arrays, we may have to define UDFs to process those elements.

In [46]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

def nonzerolist(l):
    return len(l)>0

nonzerolist_ = F.udf(nonzerolist, T.BooleanType())

df.limit(1000).filter(nonzerolist_('entities.urls')) \
    .select('user.id', 'user.name', 'entities').take(1)

[Row(id=578292128, name=u'WatchesLOVEYou', entities=Row(hashtags=[], media=None, urls=[Row(display_url=u'\u2026ttnauerquartzwatch.watchesloveyou.com/invicta-mens-7\u2026', expanded_url=u'http://wittnauerquartzwatch.watchesloveyou.com/invicta-mens-7185-signature-collection-russian-diver-black-ion-plated-chronograph-watch-cheap-price-sale/', indices=[92, 112], url=u'http://t.co/iiZIrW1b')], user_mentions=[]))]